# Códigos para geração dos arquivos CSV

In [ ]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import csv
import sys
import pandas as pd

osm_file = 'map'

d = [] # lista dos elementos das tags nodes
e = [] # lista dos elementos das tags ways
f = [] # lista dos elementos nodes_tags
g = [] # lista dos elementos ways_tags
h = [] # lista dos elementos ways_nodes

def shape_element(filename):
    tree = ET.parse(filename)
    element = tree.getroot()
    
    
    for el in element.iter():
        if el.tag == "node":
            nodes = {}
            for item in el.attrib:
                nodes_list = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
                if item in nodes_list:
                    nodes[item] = el.get(item)
                    #pprint.pprint(nodes)
            d.append(nodes)
            nodes = {}
            
            with open('nodes.csv', 'w', encoding='utf-8') as csv_file:
                nodes_list = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
                writer = csv.DictWriter(csv_file, fieldnames = nodes_list)
                writer.writeheader()
                for dic in d:
                    data = {key: value for key, value in dic.items() if key in nodes_list}
                    writer.writerow(data)
              
    
    for el in element.iter():
        if el.tag == 'way':
            ways = {}
            for item in el.attrib:
                ways_list = ["id", "user","uid", "version", "changeset", "timestamp"]
                if item in ways_list:
                    ways[item] = el.get(item)
            e.append(ways)
            ways = {}
            
            with open('ways.csv', 'w', encoding='utf-8') as csv_file:
                ways_list = ["id", "user","uid", "version", "changeset", "timestamp"]
                writer = csv.DictWriter(csv_file, fieldnames = ways_list)
                writer.writeheader()
                for dic in e:
                    data = {key: value for key, value in dic.items() if key in ways_list}
                    writer.writerow(data)
                     
            
            ways_tags = {}
            for i in el.iter('tag'):
                ways_tags['key'] = i.get('k')
                ways_tags['value'] = i.get('v')
                ways_tags['type'] = el.tag
                g.append(nodes_tags)
                nodes_tags = {}
            
    
            with open('ways_tags.csv', 'w', encoding='utf-8') as csv_file:
                fieldnames = ['key', 'value', 'type']
                writer = csv.DictWriter(csv_file, fieldnames = fieldnames)
                writer.writeheader()
                for dic in g:
                    data = {key: value for key, value in dic.items() if key in fieldnames}
                    writer.writerow(data)
                        
            ways_nodes = {}
            for i in el.iter('tag'):
                if i.tag == 'nd':
                    ways_nodes['position'] = i.get('ref')
                    h.append(ways_nodes)
                    ways_nodes = {}
            
            with open('ways_nodes.csv', 'w', encoding='utf-8') as csv_file:
                fieldnames = ['position']
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                for dic in h:
                    data = {key: value for key, value in dic.items() if key in fieldnames}
                    writer.writerow(data)
                     
    
shape_element(osm_file)

In [11]:
import csv
import sqlite3

con = sqlite3.connect('data_wrangling_sqlite.db') #nome do banco de dados

cur = con.cursor()

with open('nodes_tags.csv', 'r', encoding='utf-8') as file:
    dr = csv.DictReader(file) # nome do arquivo csv
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
    # i['nome da coluna']

cur.executemany("INSERT INTO nodes_tags VALUES (?,?,?,?);", to_db)# ? = número de colunas

con.commit()

cur.close()

con.close()


In [3]:
import csv
import sqlite3

con = sqlite3.connect('data_wrangling_sqlite.db', timeout=60000, isolation_level=None) #nome do banco de dados

cur = con.cursor()

with open('nodes.csv', 'r', encoding='utf-8') as file:
    dr = csv.DictReader(file) # nome do arquivo csv
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    # i['nome da coluna']

con.execute("PRAGMA busy_timeout = 60000")   # 60 s

cur.executemany('INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);', to_db)# ? = número de colunas

con.commit()

cur.close()

con.close()

In [12]:
import csv
import sqlite3

con = sqlite3.connect('data_wrangling_sqlite.db') #nome do banco de dados

cur = con.cursor()

with open('ways.csv', 'r', encoding='utf-8') as file:
    dr = csv.DictReader(file) # nome do arquivo csv
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    # i['nome da coluna']

cur.executemany("INSERT INTO ways VALUES (?,?,?,?,?,?);", to_db)# ? = número de colunas

con.commit()

cur.close()

con.close()


In [8]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import csv
import sys
import pandas as pd

osm_file = 'map'

d = [] # lista dos elementos das tags nodes
e = [] # lista dos elementos das tags ways
f = [] # lista dos elementos nodes_tags
g = [] # lista dos elementos ways_tags
h = [] # lista dos elementos ways_nodes



def shape_element(filename):
    tree = ET.parse(filename)
    element = tree.getroot()
    
    nodes_tags = {}
    for el in element.iter():
        if el.tag == "node":
            
            for e in el.iter('tag'):
                nodes_tags['id'] = el.get('id')
                nodes_tags['key'] = e.get('k')
                nodes_tags['value'] = e.get('v')
                nodes_tags['type'] = el.tag
                f.append(nodes_tags)
                nodes_tags = {}
                
            with open('nodes_tags.csv', 'w', encoding='utf-8') as csv_file:
                fieldnames = ['id', 'key', 'value', 'type']
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                for dic in f:
                    data = {key: value for key, value in dic.items() if key in fieldnames}
                    writer.writerow(data)            
shape_element(osm_file)        

In [31]:
import csv
import sqlite3

con = sqlite3.connect('data_wrangling_sqlite.db') #nome do banco de dados

cur = con.cursor()

with open('ways_nodes.csv', 'r', encoding='utf-8') as file:
    dr = csv.DictReader(file) # nome do arquivo csv
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]
    # i['nome da coluna']

cur.executemany("INSERT INTO ways_nodes VALUES (?,?,?);", to_db)# ? = número de colunas

con.commit()

cur.close()

con.close()

In [32]:
import csv
import sqlite3

con = sqlite3.connect('data_wrangling_sqlite.db') #nome do banco de dados

cur = con.cursor()

with open('ways_tags.csv', 'r', encoding='utf-8') as file:
    dr = csv.DictReader(file) # nome do arquivo csv
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
    # i['nome da coluna']

cur.executemany("INSERT INTO ways_tags VALUES (?,?,?,?);", to_db)# ? = número de colunas

con.commit()

cur.close()

con.close()

# Código para obtenção de amostra dos dados

In [40]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "map"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write(b'<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(b'</osm>')